# GBDT算法 Gradient Boosting Decision Tree

GBDT也是集成学习Boosting的一种算法，但是却和传统的Adaboost有很大的不同。在GBDT的迭代中，假设我们前一轮迭代得到的强学习器是$f_{t-1}(x)$，损失函数是$L(y,f_{t-1}(x))$，迭代的目标是找到一个`CART`回归树模型的弱学习器$h_t(x)$，让本轮的损失函数$L(y,f_t(x)) = L(y,f_{t-1}(x)+h_t(x))$`最小`。即每次迭代找到的决策树要让损失函数值更小。

Xgboost（Extreme）在GBDT的基础上增加了正则化项来控制模型的复杂度，提高模型泛化能力，并且可以选择更多的不限于CART的基础模型。

随即森林是构造做个基础的决策树模型，`并行`训练多个决策树模型，将各个决策树模型的最终结果取均值或则最终的决策结果，而Xgboost是在决策树模型中，先构造第一棵树，再依次构造并加入其他的树，是一个`串行`算法。

> 有科学家证明决策树模型是一个很好的基础模型。

## Xgboost 算法

Xgboost损失函数最终为：

$$
    L_t=\sum_{i=1}^mL(y_i, f_{t-1}(x_i)+ h_t(x_i)) + \gamma J + \frac{\lambda}{2}\sum_{j=1}^Jw_{tj}^2
$$

其中，L是损失函数，可以是平方损失函数或者logistics损失函数等，J为决策树t的叶子节点的个数，w是叶子节点的最优值，参数$\gamma$是限制叶子节点个数，$\lambda$是正则化惩罚力度。

利用二阶泰勒展开，得到一个近似解，一阶导数和二阶导数转化合并为$g_i$和$h_i$：

$$
    L_t \approx \sum_{i=1}^m( L(y_i, f_{t-1}(x_i)) + g_{ti}h_t(x_i) + \frac{1}{2} h_{ti} h_t^2(x_i)) +  \gamma J + \frac{\lambda}{2}\sum_{j=1}^Jw_{tj}^2
$$

除去损失函数进一步简化目标函数，再将原本按样本遍历，转化为按叶子节点遍历：

$$
    L_t  =  \sum_{j=1}^J [G_{tj}w_{tj} + \frac{1}{2}(H_{tj}+\lambda)w_{tj}^2] + \gamma J
$$

其中$G_{tj}$和$H_{tj}$利用一阶和二阶导函数计算得到。

到此为止得到了最终的损失函数，那么最小化损失函数就可以得到叶子区域的最优解$w_{tj}$。

对损失函数求导且令其为0，得到最优解$w_{tj}$：

$$
    w_{tj} = - \frac{G_{tj}}{H_{tj} + \lambda}
$$

注意到在$w_{tj}$取最优解的时候，原损失函数对应的表达式为：

$$
    L_t = -\frac{1}{2} \sum_{j=1}^J\frac{G_{tj}^2}{H_{tj} + \lambda} +\gamma J
$$

目标函数中的$G/(H+\lambda)$部分，表示着每一个叶子节点对当前模型损失的贡献程度，得到Gain的计算表达式：

$$
    \max \frac{1}{2}\frac{G_L^2}{H_L + \lambda} + \frac{1}{2}\frac{G_R^2}{H_R+\lambda}  - \frac{1}{2}\frac{(G_L+G_R)^2}{H_L+H_R+ \lambda} - \gamma
$$


## Xgboost算法流程

> 输入是训练集样本$I=\{(x_1,y_1),(x_2,y_2),\dots,(x_m,y_m)\}$，最大迭代次数T, 损失函数L，正则化系数λ,γ。
> 
> 输出是强学习器f(x)

对迭代轮数$t=1,2,\dots,T$有：
1. 计算第i个样本$(i=1,2,\dots,m)$在当前轮基于$f_{t-1}(x_i)$的一阶导数$g_{ti}$和二阶导数$h_{ti}$的损失函数L，计算所有样本的一阶导数和$G_t = \sum_{i=1}^m g_{ti}$,二阶导数和$H_t = \sum_{i=1}^m h_{ti}$。
2. 基于当前节点尝试分裂决策树，默认分数score=0，G和H为当前需要分裂的节点的一阶二阶导数之和。

   对特征序号$k=1,2,\dots,K$:

   1. $G_L=0, H_L=0$
   2. 1. 将样本`按特征k里的值从小到大排列`，线性扫描样本，依次计算当前样本放入左子树后，左右子树一阶和二阶导数和：
      $$
          G_L = G_L+ g_{ti}, G_R=G-G_L \\
          H_L = H_L+ h_{ti}, H_R=H-H_L
      $$
   2. 2. 根据`贪心算法`，尝试更新最大的分数：
      $$
          score = max(score, \frac{1}{2} \frac{G_L^2}{H_L + \lambda} + \frac{1}{2}\frac{G_R^2}{H_R+\lambda}  - \frac{1}{2} \frac{(G_L+G_R)^2}{H_L+H_R+ \lambda} -\gamma )
      $$
3. 基于最大score对应的划分特征和特征值分裂子树。
4. 如果最大score为0，则当前决策树建立完毕，计算所有叶子区域的$w_{tj}$, 得到弱学习器$h_t(x)$，更新强学习器$f_t(x)$,进入下一轮弱学习器迭代.如果最大score不是0，则转到第2步继续尝试分裂决策树。

## xgboost特点

* 优势
    1. 而 Xgboost 在 GBDT 的基础上对损失函数进行了二阶泰勒展开，一方面增加精度，另一方面二阶泰勒展开可以近似大量损失函数，从而进行使用；
    2. Xgboost 不仅支持 CART 还支持其他基础分类器；
    3. 在目标函数中加入了正则项，防止过拟合，控制模型复杂度；
    4. 对于特征的值有缺失的样本，Xgboost 采用的稀疏感知算法可以自动学习出它的分裂方向，很好的自动处理缺失值；
    5. 在特征粒度上进行并行计算（Xgboost 耗时主要在对特征的值进行排序）
* 劣势
    1. 预排序过程的空间复杂度过高，不仅需要存储特征值，还需要存储特征对应样本的梯度统计值的索引，相当于消耗了两倍的内存。

> 可以看出，Xgboost 的缺点相对于优势来说很小，也是xgboost被称为kaggle神器的原因。

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
# %matplotlib widget
plt.rcParams["font.sans-serif"] = 'SimHei'  # 中文问题
plt.rcParams["axes.unicode_minus"] = False  # 负号问题
%config InlineBackend.figure_format = 'svg'

In [2]:
from sklearn.datasets import make_classification
# X为样本特征，y为样本类别输出， 共10000个样本，每个样本20个特征，输出有2个类别，没有冗余特征，每个类别一个簇
X, y = make_classification(n_samples=1000, n_features=20,
                           n_redundant=0, n_clusters_per_class=1, n_classes=2, flip_y=0.1)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

### XGBoost的框架参数

最重要的是3个参数: booster，n_estimators和objectve。
1. booster决定了XGBoost使用的弱学习器类型，可以是默认的gbtree, 也就是CART决策树，还可以是线性弱学习器gblinear以及DART。一般来说，我们使用gbtree就可以了，不需要调参。
2. n_estimators参数很重要，它关系到XGBoost模型的复杂度，代表决策树弱学习器的个数。n_estimators太小，容易欠拟合，n_estimators太大，模型会过于复杂，一般需要调参选择一个适中的数值。
3. objective代表了我们要解决的问题是分类还是回归，或其他问题，以及对应的损失函数。具体可以取的值很多，一般我们只关心在分类和回归的时候使用的参数。
    > 在回归问题objective一般使用reg:squarederror ，即MSE均方误差。二分类问题一般使用binary:logistic, 多分类问题一般使用multi:softmax。

### 弱学习器的参数
这里我们只讨论使用默认的gbtree，要调参的参数主要是决策树的相关参数，具体的取值一般要网格搜索调参。 

1. max_depth：控制树结构的深度，数据少或者特征少的时候可以不管这个值。如果模型样本量多，特征也多的情况下，需要限制这个最大深度。
2. min_child_weight：最小的子节点权重阈值，如果某个树节点的权重小于这个阈值，则不会再分裂子树。这里树节点的权重使用的是该节点所有样本的二阶导数的和，即XGBoost之前所说的$H_{tj}$:
3. gamma：XGBoost的决策树分裂所带来的损失减小阈值。也就是在尝试树结构分裂时，得到的最大化score需要大于gamma，才能继续分裂子树。
4. subsample：子采样参数，这个也是不放回抽样。选择小于1的比例可以减少方差，即防止过拟合，但是会增加样本拟合的偏差，因此取值不能太低。
5. colsample_bytree/colsample_bylevel/colsample_bynode：这三个参数都是用于特征采样的，默认都是不做采样，即使用所有的特征建立决策树。
    * colsample_bytree：控制整棵树的特征采样比例
    * colsample_bylevel：控制某一层的特征采样比例
    * colsample_bynode：控制某一个树节点的特征采样比例
6. reg_alpha/reg_lambda：这2个是XGBoost的正则化参数。reg_alpha是L1正则化系数，reg_lambda是L2正则化系数，

> 上面这些参数都是需要调参的，不过一般先调前三个。如果发现有过拟合的情况下，再尝试调后面几个参数。

### 其他参数

XGBoost还有一些其他的参数需要注意：
* learning_rate：控制每个弱学习器的权重缩减系数，较小的learning_rate意味着我们需要更多的弱学习器的迭代次数。通常我们用步长和迭代最大次数一起来决定算法的拟合效果。所以这两个参数n_estimators和learning_rate要一起调参才有效果。当然也可以先固定一个learning_rate ，然后调完n_estimators，再调完其他所有参数后，最后再来调learning_rate和n_estimators。
* importance_type则可以查询各个特征的重要性程度。可以选择“gain”, “weight”, “cover”, “total_gain” 或者 “total_cover”。最后可以通过调用booster的get_score方法获取对应的特征权重。“weight”通过特征被选中作为分裂特征的计数来计算重要性，“gain”和“total_gain”则通过分别计算特征被选中做分裂特征时带来的平均增益和总增益来计算重要性。“cover”和 “total_cover”通过计算特征被选中做分裂时的平均样本覆盖度和总体样本覆盖度来来计算重要性。

In [4]:
import xgboost as xgb

sklearn_model_new = xgb.XGBClassifier(
    max_depth=5, learning_rate=0.5, verbosity=1, objective='binary:logistic', random_state=1)

In [5]:
sklearn_model_new.fit(X_train, y_train, early_stopping_rounds=10,
                      eval_metric='error', eval_set=[(X_test, y_test)])

[0]	validation_0-error:0.12333
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.11333
[2]	validation_0-error:0.10667
[3]	validation_0-error:0.10667
[4]	validation_0-error:0.10667
[5]	validation_0-error:0.11667
[6]	validation_0-error:0.11333
[7]	validation_0-error:0.11000
[8]	validation_0-error:0.11000
[9]	validation_0-error:0.11333
[10]	validation_0-error:0.11333
[11]	validation_0-error:0.11667
[12]	validation_0-error:0.11333
Stopping. Best iteration:
[2]	validation_0-error:0.10667



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=1)

In [6]:
from sklearn.metrics import accuracy_score
# 训练集预测
pred_train_sklearn = sklearn_model_new.predict(X_train)
pred_train_sklearn[pred_train_sklearn > 0.5] = 1
pred_train_sklearn[pred_train_sklearn <= 0.5] = 0

print(accuracy_score(y_train, pred_train_sklearn))

0.9471428571428572


In [7]:
pred_test_sklearn = sklearn_model_new.predict(X_test)
pred_test_sklearn[pred_test_sklearn > 0.5] = 1
pred_test_sklearn[pred_test_sklearn <= 0.5] = 0

print(accuracy_score(y_test, pred_test_sklearn))

0.8933333333333333


### 网格搜索调参

In [8]:
from sklearn.model_selection import GridSearchCV

gsCv = GridSearchCV(sklearn_model_new, {'max_depth': [
                    2, 3, 4, 5, 6], 'n_estimators': [5, 10, 20, 50]})
gsCv.fit(X_train, y_train)
print(gsCv.best_score_)
print(gsCv.best_params_)

0.9042857142857145
{'max_depth': 2, 'n_estimators': 5}


In [9]:
sklearn_model_new2 = xgb.XGBClassifier(
    max_depth=2, n_estimators=5, verbosity=1, objective='binary:logistic', random_state=1)
gsCv2 = GridSearchCV(sklearn_model_new2, {'learning_rate': [0.3, 0.5, 0.7]})
gsCv2.fit(X_train, y_train)
print(gsCv2.best_score_)
print(gsCv2.best_params_)

0.9042857142857145
{'learning_rate': 0.5}


In [10]:
sklearn_model_last = xgb.XGBClassifier(
    max_depth=2, n_estimators=5, learning_rate=0.5, verbosity=1, objective='binary:logistic', random_state=1)
sklearn_model_last.fit(X_train, y_train, early_stopping_rounds=10,
                       eval_metric='error', eval_set=[(X_test, y_test)])
# 即准确率为 1 - validation_0-error

[0]	validation_0-error:0.10000
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.10000
[2]	validation_0-error:0.10000
[3]	validation_0-error:0.10000
[4]	validation_0-error:0.11333


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=5, n_jobs=0, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=1)